# Redes Neuronales - Trabajo Práctico N° 2 - Ejercicio 1 - Regresión Logística
# Notebook #5: K Folding + HyperOpt en MLP
En esta notebook se emplea K Folding para la optimización de un modelo MLP con HyperOpt.
### Integrantes del grupo
* Kammann, Lucas Agustín
* Gaytan, Joaquín Oscar

In [1]:
import numpy as np

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Read database from .csv
df = pd.read_csv('../../databases/diabetes.csv', delimiter=',')

# Show first rows of data
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
from src.mlp_helper import run_model_with_kfold

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [5]:
train_dict, valid_dict, test_dict = run_model_with_kfold(df=df, test_size=0.2, folds=5, random_state=15,
                                            optimizer='adam',
                                            loss='binary_crossentropy',
                                            momentum=0,
                                            rho=0,
                                            beta_1=0,
                                            beta_2=0,
                                            learning_rate=0.1,
                                            decay_rate=0.5,
                                            batch_size=64,
                                            epochs=200,
                                            patience=20,
                                            min_delta=0.001,
                                            tensorboard_on=False,
                                            summary_on=False,
                                            tag='EX1',
                                            hidden_layers=1,
                                            units_per_layer=2,
                                            hidden_layer_activation='sigmoid',
                                            regularizer='l2',
                                            regularizer_lambda=1e-4,
                                            dropout_rate=0.0,
                                            use_batch_normalization=False)

In [6]:
auc_train = train_dict['auc']
auc_valid = valid_dict['auc']
auc_test = test_dict['auc']
print(f'[AUC] Train: {auc_train:.4f} - Valid: {auc_valid:.4f} - Test: {auc_test:.4f}')

[AUC] Train: 0.8421 - Valid: 0.8326 - Test: 0.8052


# Using HyperOpt

In [7]:
import hyperopt

In [8]:
from hyperopt.pyll import scope

In [9]:
from src import mlp_helper
from src.helper import print_metrics

In [ ]:
def objective(kwargs):
    """ Objective function for the hyperopt algorithm. """
    
    # Setting some fixed values
    kwargs['tensorboard_on'] = False
    kwargs['summary_on'] = False
    kwargs['epochs'] = 500
    kwargs['batch_size'] = 64
    kwargs['optimizer'] = 'adam'
    kwargs['beta_1'] = 0.9
    kwargs['beta_2'] = 0.9
    kwargs['dropout_rate'] = 0
    
    # Running the model with the given hyperparameters and retrieving the test set performance
    # WARNING! We're using the same valid set for both valid and test, but can be ignored, it does not affect
    # because it was something we had to do to reuse the function...
    _, eval_valid, _ = mlp_helper.run_model_with_kfold(df=df, folds=5, test_size=0.2, random_state=15, **kwargs)
    auc_valid = eval_valid['auc']
    return (-1*auc_valid)

In [45]:
space = {
    'use_batch_normalization': hyperopt.hp.choice('use_batch_normalization', [False, True]),
    'learning_rate': hyperopt.hp.choice('learning_rate', [1.0, 0.5, 0.1, 0.01]),
    'hidden_layers': scope.int(hyperopt.hp.quniform('hidden_layers', 1, 10, 1)),
    'units_per_layer': scope.int(hyperopt.hp.quniform('units_per_layer', 2, 100, 8)),
    'hidden_layer_activation': hyperopt.hp.choice('hidden_layer_activation', ['relu', 'tanh','sigmoid']),
    'regularizer': hyperopt.hp.choice('regularizer', ['l1', 'l2']),
    'regularizer_lambda': hyperopt.hp.choice('regularizer_lambda', [1, 1e-2, 1e-3, 1e-5, 1e-7])
}

In [46]:
algorithm = hyperopt.tpe.suggest

In [47]:
trials = hyperopt.Trials()
best = hyperopt.fmin(objective, space, algo=algorithm, max_evals=200, trials=trials)

  0%|▏                                              | 1/200 [00:13<45:42, 13.78s/trial, best loss: -0.8339706355993766]

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:158: RuntimeWarning: Mean of empty slice
  train_dict['ppv'] = np.nanmean(train_dict['ppv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:164: RuntimeWarning: Mean of empty slice
  valid_dict['ppv'] = np.nanmean(valid_dict['ppv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:170: RuntimeWarning: Mean of empty slice
  test_dict['ppv'] = np.nanmean(test_dict['ppv'])



  2%|▋                                            | 3/200 [01:13<1:25:33, 26.06s/trial, best loss: -0.8339706355993766]

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:158: RuntimeWarning: Mean of empty slice
  train_dict['ppv'] = np.nanmean(train_dict['ppv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:164: RuntimeWarning: Mean of empty slice
  valid_dict['ppv'] = np.nanmean(valid_dict['ppv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:170: RuntimeWarning: Mean of empty slice
  test_dict['ppv'] = np.nanmean(test_dict['ppv'])



  5%|██▏                                         | 10/200 [04:17<1:30:55, 28.71s/trial, best loss: -0.8339706355993766]

...

 98%|█████████████████████████████████████████▉ | 195/200 [1:21:14<02:20, 28.01s/trial, best loss: -0.8651503242130782]

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:159: RuntimeWarning: Mean of empty slice
  train_dict['npv'] = np.nanmean(train_dict['npv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:165: RuntimeWarning: Mean of empty slice
  valid_dict['npv'] = np.nanmean(valid_dict['npv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:171: RuntimeWarning: Mean of empty slice
  test_dict['npv'] = np.nanmean(test_dict['npv'])



 98%|██████████████████████████████████████████▏| 196/200 [1:21:50<02:01, 30.27s/trial, best loss: -0.8651503242130782]

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:164: RuntimeWarning: Mean of empty slice
  valid_dict['ppv'] = np.nanmean(valid_dict['ppv'])

C:\Users\Joaco\Desktop\TP2 Redes Neuronales\nn-tp2\notebooks\ej1\src\mlp_helper.py:170: RuntimeWarning: Mean of empty slice
  test_dict['ppv'] = np.nanmean(test_dict['ppv'])



100%|███████████████████████████████████████████| 200/200 [1:23:37<00:00, 25.09s/trial, best loss: -0.8651503242130782]


In [48]:
best

{'hidden_layer_activation': 0,
 'hidden_layers': 4.0,
 'learning_rate': 2,
 'regularizer': 1,
 'regularizer_lambda': 1,
 'units_per_layer': 56.0,
 'use_batch_normalization': 1}

In [38]:
train_dict, valid_dict, test_dict = run_model_with_kfold(df=df, test_size=0.2, folds=5, random_state=15,
                                            optimizer='adam',
                                            loss='binary_crossentropy',
                                            beta_1=0.9,
                                            beta_2=0.9,
                                            learning_rate=0.01,
                                            batch_size=64,
                                            epochs=500,
                                            tensorboard_on=False,
                                            summary_on=False,
                                            tag='best_kfold',
                                            hidden_layers=5,
                                            units_per_layer=72,
                                            hidden_layer_activation='relu',
                                            dropout_rate=0.0,
                                            use_batch_normalization=False)

In [39]:
print_metrics(train_dict, valid_dict, test_dict)

[AUC] Train: 0.8814 - Valid: 0.8533 - Test: 0.7997


In [49]:
train_dict, valid_dict, test_dict = run_model_with_kfold(df=df, test_size=0.2, folds=5, random_state=15,
                                            optimizer='adam',
                                            loss='binary_crossentropy',
                                            beta_1=0.9,
                                            beta_2=0.9,
                                            learning_rate=0.1,
                                            batch_size=64,
                                            regularizer='l2',
                                            regularizer_lambda=1e-2,
                                            epochs=500,
                                            tensorboard_on=False,
                                            summary_on=False,
                                            tag='best_kfold',
                                            hidden_layers=4,
                                            units_per_layer=56,
                                            hidden_layer_activation='relu',
                                            dropout_rate=0.0,
                                            use_batch_normalization=True)

In [50]:
print_metrics(train_dict, valid_dict, test_dict)

[AUC] Train: 0.8768 - Valid: 0.8580 - Test: 0.7952
